In [ ]:
"""
created on: Wed Jan 15 16:58:39 2020
Group 7
@author: T.F
"""

In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from PIL import Image
from tqdm import tqdm_notebook as tqdm
# keras : librairie de deep learning
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, InputLayer, ReLU, Dense, Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from shutil import copyfile
import PIL
from PIL import Image

In [3]:
df = pd.read_csv(r'C:\Users\ups\Downloads\g7_SEATGURU_annotate.csv', sep =";")

In [4]:
def B737():
    list=[]
    for i,j in enumerate(df['name']):
        if df['aircraft_type'][i] == '737' and df['view'][i] == 'Int'  : 
            list.append(j)
    return list
B737 = B737()

In [5]:
def B747():
    list=[]
    for i,j in enumerate(df['name']):
        if df['aircraft_type'][i] == '747' and df['view'][i] == 'Int'  : 
            list.append(j)
    return list
B747 = B747()

In [6]:
def B757():
    list=[]
    for i,j in enumerate(df['name']):
        if df['aircraft_type'][i] == '757' and df['view'][i] == 'Int'  : 
            list.append(j)
    return list
B757 = B757()

In [7]:
def B777():
    list=[]
    for i,j in enumerate(df['name']):
        if df['aircraft_type'][i] == '777' and df['view'][i] == 'Int'  : 
            list.append(j)
    return list
B777 = B777()

In [ ]:
path2 = r"C:\Users\ups\Desktop\Interpromo2020\Interpromo2020\All Data\ANALYSE IMAGE\data\757\\"
for pict in B757:
    copyfile(path + pict, path2 + pict)

In [ ]:
path2 = r"C:\Users\ups\Desktop\Interpromo2020\Interpromo2020\All Data\ANALYSE IMAGE\data\777\\"
for pict in B777:
    copyfile(path + pict, path2 + pict)

In [ ]:
path2 = r"C:\Users\ups\Desktop\Interpromo2020\Interpromo2020\All Data\ANALYSE IMAGE\data\747\\"
for pict in B747:
    copyfile(path + pict, path2 + pict)

In [ ]:
path2 = r"C:\Users\ups\Desktop\Interpromo2020\Interpromo2020\All Data\ANALYSE IMAGE\data\737\\"
for pict in B737:
    copyfile(path + pict, path2 + pict)

In [ ]:
shape = (224, 224)
nb_win = 3  # number of windows
greys = False
by_aircraft_type = False  # if False: by aircraft manufacturer
imgs_per_class = 950
nb_to_gen = 1000

# Define Data Generator
# New images: rotate, zoom, shift
img_gen = ImageDataGenerator(rotation_range=10,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.09)  # other parameters: vertical_flip, horizontal_flip

def datagen_class(imgs: list, batch_size: int, gen):
    X_ref = np.array(imgs)
    new_imgs = gen.flow(X_ref, batch_size=batch_size).next()
    return new_imgs


if by_aircraft_type:
    # By aircraft type
    classes = flat_list(aircraft_types)
    ([ B737_win, B747_win, B757_win, B777_win],
     [ B737_gen, B747_gen, B757_gen, B777_gen]) = two_step_generator(classes,
                                                                    all_imgs_flat_list, 
                                                                    imgs_per_class, shape, 
                                                                    nb_win, greys, 
                                                                    nb_to_gen, img_gen)
    
else:
    # Airbus vs. Boeing
    classes = ['Boeing']
    ([Boeing_win], [Airbus_gen, Boeing_gen]) = two_step_generator(classes, all_imgs_flat_list, imgs_per_class, shape, 
                                                                              nb_win,greys, nb_to_gen, img_gen)

In [8]:
def flat_list(list_: list) -> list:
   
    return [item for sublist in list_ for item in sublist]


In [14]:
project_path = r"C:\Users\ups\Desktop\Interpromo2020\Interpromo2020\All Data\\"
data_path = r"\ANALYSE IMAGE\BOEING\\"


# Get folders list (one item per aircraft manufacturer)
folders_list = os.listdir(project_path + data_path)  # 2 folders: Airbus and Boeing
folders_list

['SEATGURU BOEING']

In [19]:
project_path = r"C:\Users\ups\Desktop\Interpromo2020\Interpromo2020\All Data\\"
data_path = r"\ANALYSE IMAGE\BOEING\\"


# Get folders list (one item per aircraft manufacturer)
folders_list = os.listdir(project_path + data_path)  # 2 folders: Airbus and Boeing

# Desired aircraft types
Boeing_aircraft_types = ['737', '747', '757', '777']
aircraft_types = list([Boeing_aircraft_types])

# Lists: all images paths and names
all_imgs_list = list()
for k in range(len(folders_list)):
    
    folder = folders_list[k]
    aircraft_types_man = aircraft_types[k]

    # For each folder (aircraft type), get list of all images names (with path)
    for j in range(len(aircraft_types_man)):
        img_list = os.listdir(project_path + data_path + folder + '\\' + aircraft_types_man[j])
        img_list = [folder + '\\' + aircraft_types_man[j]  + '\\' + img_list[k] for k in range(len(img_list))]
        all_imgs_list.append(img_list)
        
# Flat list
all_imgs_flat_list = flat_list(all_imgs_list)

In [74]:
folders_list

['SEATGURU BOEING']

In [ ]:
path_list = []
for i in len(all_imgs_flat_list):
    path_list = r"C:\Users\ups\Desktop\Interpromo2020\Interpromo2020\All Data\ANALYSE IMAGE\BOEING\SEATGURU BOEING\737" + all_imgs_flat_list[i]
    

In [23]:
def get_imgs(paths_list: list) -> list:
    
    """
    From a list of images paths, get a list of PIL images
    """
    
    imgs_list = [Image.open(project_path + data_path + paths_list[i]) for i in range(len(paths_list))]
    
    return imgs_list


In [35]:
def create_windows_one_img(shape: tuple, img: PIL.Image, nb_win: int, greys: bool) -> list:
    
    """
    Parameters:
        shape: desired shape (height, width)
        img: reference image
        nb_win: desired number of windows
    
    Out:
        arrs_crop: arrays (representing images) created by cropping the reference image
    
    """

    img_arr = np.array(img)
    
    if greys is True:
        imgs_arr = np.mean(img_arr, axis = 2)
    
    if img_arr.shape[0] <= img_arr.shape[1]:
        # Choose fixed height, calculate width (to keep height to width ratio), resize
        coef = img_arr.shape[0] / shape[0]
        temp_width = int(img_arr.shape[1] / coef)
        resized_img = img.resize((temp_width, shape[0]), Image.BILINEAR)
    
    else:
        # Choose fixed width, calculate height (to keep height to width ratio), resize
        coef = img_arr.shape[1] / shape[1]
        temp_height = int(img_arr.shape[0] / coef)
        resized_img = img.resize((shape[1], temp_height), Image.BILINEAR)
    

    # Convert resized image to array
    if greys is True:
        resized_arr = np.mean(np.array(resized_img), axis = 2)
        
    else:
        resized_arr = np.array(resized_img)

    # Crop chosen number of windows
    lag = int((int(resized_arr.shape[1]) - shape[1]) / nb_win -1)
    bounds = np.arange(0, int(resized_arr.shape[1]) - shape[1], lag)

    arrs_crop = list()
    for k in bounds:
        if greys:
            cropped_img = resized_arr[:, k:k+shape[1]].reshape(shape[0], shape[1], 1)
        else:
            cropped_img = resized_arr[:, k:k+shape[1]]
        arrs_crop.append(cropped_img)
        
    return arrs_crop


In [52]:
def two_step_generator(classes: list, paths_list: list, imgs_per_class: int, shape: tuple,
                       nb_win: int, greys: bool, nb_to_gen: int, img_gen: ImageDataGenerator) -> list:
    
    """
    From a list of reference images, performs a 2-step transformation to generate new images
    of a chosen shape.
    Step 1: Resizes images keeping height to width ratio, and generates a set of cropped images (sliding windows)
    Step 2: Generates new images from Step 1 images, by applying rotations, zooms and shifts
    
    Parameters:
        classes: list of desired classes, e.g.: ['Airbus', 'Boeing'], or a list of aircraft types
        paths_list: list of images paths
        imgs_per_class: desired number of images per class from Step 1 (resize + crop)
        shape: desired shape (height, width)
        nb_win: number of windows
        greys: True for grey scale, False for colour scale
        nb_to_gen: desired number of images per class from data generator
        img_gen: ImageGenerator object
  
    Out:
        datagen: list of generated arrays (representing images)
  
    """
    
    datawin = list()    
    datagen = list()
    
    for class_ in classes:
        print(class_)
        
        # Images paths list
        class_imgs_path = [paths_list[k] for k in range(len(paths_list)) if class_ in paths_list[k]]

        # Randomly choose images
        class_imgs_subset = np.random.choice(class_imgs_path, size=imgs_per_class, replace=False)

        # Get images
        class_imgs = get_imgs(class_imgs_subset)

        # Step 1: resize and crop on sliding windows
        class_new_imgs = create_windows_imgs(class_imgs, shape=shape, nb_win=nb_win, greys=greys)
        class_new_imgs = np.array(flat_list(class_new_imgs))
        datawin.append(class_new_imgs)
    
        # Step 2: DataGenerator
        class_datagen = datagen_class(class_new_imgs, nb_to_gen, img_gen)
        class_datagen = class_datagen.astype(int)
        
        print(class_datagen.shape)

        datagen.append(class_datagen)
        
    return datawin, datagen

In [53]:
def create_windows_imgs(init_imgs: list, shape: tuple, nb_win:int, greys: bool) -> list:
    
    """
    Parameters:
        init_imgs: list of reference images
        shape: desired shape (height, width)
        nb_win: desired number of windows
    
    Out:
        new_imgs_all: list of all new arrays (representing images), created by cropping the reference images
    
    """
    
    new_imgs_all = list()
    
    for k in range(len(init_imgs)):
        img = init_imgs[k]
        new_imgs = create_windows_one_img(shape=shape, img=img, nb_win=nb_win, greys=greys)
        new_imgs_all.append(new_imgs)
                
    return new_imgs_all



In [77]:
# Choose parameters
shape = (224, 224)
nb_win = 3  # number of windows
greys = False
by_aircraft_type = True  # if False: by aircraft manufacturer
imgs_per_class = 44
nb_to_gen = 300

# Define Data Generator
# New images: rotate, zoom, shift
img_gen = ImageDataGenerator(rotation_range=10,
                             width_shift_range=0.1,
                             height_shift_range=0.1,
                             zoom_range=0.09)  # other parameters: vertical_flip, horizontal_flip

def datagen_class(imgs: list, batch_size: int, gen):
    X_ref = np.array(imgs)
    new_imgs = gen.flow(X_ref, batch_size=batch_size).next()
    return new_imgs


if by_aircraft_type:
    # By aircraft type
    classes = flat_list(aircraft_types)
    ([B737_win, B747_win, B757_win, B777_win],
     [B737_gen, B747_gen, B757_gen, B777_gen]) = two_step_generator(classes,
                                                                    all_imgs_flat_list, 
                                                                    imgs_per_class, shape, 
                                                                    nb_win, greys, 
                                                                    nb_to_gen, img_gen)
    
else:
    # Airbus vs. Boeing
    classes = ['737','747','757','777']
    ([Boeing_win, Boeing_gen]) = two_step_generator(classes, all_imgs_flat_list, imgs_per_class, shape, 
                                                                              nb_win,greys, nb_to_gen, img_gen)

737
(64, 224, 224, 3)
747
(68, 224, 224, 3)
757
(64, 224, 224, 3)
777
(80, 224, 224, 3)


In [99]:
def data_augmentation(train_path, rotation_range = 40, width_shift_range = .2, height_shift_range = .2, shear_range = .2, zoom_range = .2, horizontal_flip = True, nb_img = 10):
    classes = os.listdir(train_path)

    datagen = ImageDataGenerator(
           rotation_range=rotation_range,
           width_shift_range=width_shift_range,
           height_shift_range=height_shift_range,
           shear_range=shear_range,
           zoom_range=zoom_range,
           horizontal_flip=horizontal_flip,
           fill_mode='nearest')

    for classe in classes:
        picts = os.listdir(train_path + '/' + classe)
        print(classe)

        for pict in picts:
            img = Image.open(train_path + '/' + classe + '/' + pict)
            img = np.array(img)
            img = img.reshape((1,) + img.shape)
            i=0

            for batch in datagen.flow(img, batch_size=1, save_to_dir=train_path, save_prefix=classe + '/' + classe, save_format='jpeg'):
                i += 1
                if i > nb_img:
                    break  # otherwise the generator would loop indefinitely

In [101]:
data_augmentation(r"C:\Users\ups\Desktop\Interpromo2020\Interpromo2020\All Data\ANALYSE IMAGE\train")

A320
A321
A330
A350
